In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools 
import multiprocessing as mp
import numpy as np
import sys

sys.path.append('../')
from baseModels.SVM.model import SVM

In [2]:
feature_dir = '../featEngg/pssmMethods/data/featvec/'
feature_files = [f.name for f in os.scandir(feature_dir) if f.name.endswith('.csv.gz')]

In [3]:
label_file = '../data/LabelFiles/EnzymeLabelsMultiClass.csv'

In [4]:
def model_evaluate(filename,random_seed):
    df1 = pd.read_csv(feature_dir+filename,header=None)
    df2 = pd.read_csv(label_file,header=None)
    df = df1.merge(df2,on=0)
    enz_names = df[0].values
    X = df.iloc[:,1:-1].values
    y = df.iloc[:,-1].values
    if X.shape[1]<40:
        n_comp = int(0.75*X.shape[1])
    else:
        n_comp=40
    X_train, X_test, y_train, y_test,enz_train,enz_test = train_test_split(X, y,enz_names, test_size=0.25, random_state=random_seed)
    svm = SVM(X_train,X_test,y_train,y_test,verbose=False,optimize=False, pca_comp=n_comp,kern='rbf',regC=30)
    return svm.acc_train,svm.acc_test
 

In [5]:
mp.cpu_count()

8

In [6]:
pool = mp.Pool(mp.cpu_count())

In [7]:
iterable = list(itertools.product(feature_files,range(10)))

In [8]:
%%time
model_accs = list(pool.starmap(model_evaluate,iterable))

CPU times: user 9.19 ms, sys: 6.09 ms, total: 15.3 ms
Wall time: 7.58 s


In [9]:
mydict = dict()

for m,acc in zip(iterable,model_accs):
    model_name=m[0].replace('.csv.gz','')
    seed = m[1]
    if model_name not in mydict:
        mydict[model_name] = {seed:acc}
    else:
        mydict[model_name][seed] = acc

In [10]:
def get_model_stats(mname):
    train_accs = [v[0] for k,v in mydict[mname].items()]
    test_accs = [v[1] for k,v in mydict[mname].items()]
    return {'train_min':min(train_accs),'train_mean':np.mean(train_accs),
            'train_max':max(train_accs),'train_std':np.std(train_accs),
            'test_min':min(test_accs),'test_mean':np.mean(test_accs),
            'test_max':max(test_accs),'test_std':np.std(test_accs)}

In [11]:
get_model_stats('rpssm')

{'train_min': 0.813953488372093,
 'train_mean': 0.836046511627907,
 'train_max': 0.8837209302325582,
 'train_std': 0.022336479898021563,
 'test_min': 0.5172413793103449,
 'test_mean': 0.5896551724137933,
 'test_max': 0.6896551724137931,
 'test_std': 0.07625291168102062}

In [12]:
results_dict= {mname:get_model_stats(mname) for mname in mydict.keys()}
    

In [13]:
results_df = pd.DataFrame(results_dict).T

In [14]:
results_df.to_csv('../data/SimResults/pssmResults.csv',index=True)

In [15]:
results_df.sort_values('train_mean')

,train_min,train_mean,train_max,train_std,test_min,test_mean,test_max,test_std
tpc,0.604651,0.712791,1.000000,0.146627,0.413793,0.620690,0.793103,0.091233
edp,0.697674,0.737209,0.779070,0.025581,0.448276,0.606897,0.758621,0.075862
smoothed_pssm,0.767442,0.808140,0.860465,0.025074,0.413793,0.548276,0.724138,0.096859
rpssm,0.813953,0.836047,0.883721,0.022336,0.517241,0.589655,0.689655,0.076253
aatp,0.802326,0.836047,0.906977,0.038372,0.517241,0.620690,0.724138,0.055602
medp,0.825581,0.858140,0.895349,0.017863,0.517241,0.589655,0.689655,0.052182
aac_pssm,0.837209,0.859302,0.883721,0.016810,0.517241,0.637931,0.724138,0.060222
ab_pssm,0.825581,0.860465,0.883721,0.018014,0.551724,0.610345,0.724138,0.053532
eedp,0.825581,0.861628,0.895349,0.018349,0.551724,0.593103,0.689655,0.048276
pssm_cc,0.848837,0.881395,0.906977,0.017089,0.551724,0.627586,0.689655,0.057287
